In [1]:
import os
from multiprocessing import Pool

import networkx as nx
import numpy as np

from src.vrp_study.pdptw_model.pdptw_routing_manager_builder import PDRoutingManagerBuilder
from src.vrp_study.pdptw_model.routing_model import find_optimal_paths
from src.vrp_study.data_loader import parse_data
from src.vrp_study.configs import ModelConfig
from vrp_study.data_model import Tariff, Cargo, Node
from vrp_study.routing_manager import RoutingManager
from src.vrp_study.pdptw_model.solution_builder import SolutionBuilder
import pickle


In [2]:
def func(du, dv):
    return np.sqrt((du[0] - dv[0]) ** 2 + (du[1] - dv[1]) ** 2)


def calc(data):
    u, du, p2coordinate = data
    return {(u, v): func(du, dv) for v, dv in p2coordinate.items()}


def rm(
        depo: Node,
        cargos: list[Cargo],
        tariff: Tariff,
        p: Pool
) -> RoutingManager:
    p2coordinates = {}

    p2coordinates.update({
        crg.nodes[i].id: crg.nodes[i].coordinates for crg in cargos for i in range(2)
    })
    p2coordinates[depo.id] = depo.coordinates

    res = list(p.imap_unordered(calc, [(u, du, p2coordinates) for u, du in p2coordinates.items()]))
    distance_matrix = {}
    time_matrix = {}
    for r in res:
        for k, v in r.items():
            distance_matrix[k] = v
            time_matrix[k] = v

    routing_manager = PDRoutingManagerBuilder(
        distance_matrix=distance_matrix,
        time_matrix=time_matrix,
        model_config=ModelConfig(max_execution_time_minutes=1)
    )

    routing_manager.add_cargos(cargos)
    routing_manager.add_tariff(tariff)

    routing_manager.add_depo(depo)

    routing_manager = routing_manager.build()
    return routing_manager

In [3]:
def rms_calc(data):
    routing_manager, benchmark_type, name = data
    sol = find_optimal_paths(routing_manager, SolutionBuilder(
        save_graph=True, name = f'{benchmark_type}_{name}'
    ))

    sols = []
    for s in sol[0]:
        if len(s) > 0:
            sols.append([i for i in s[1:-1]])
    with open(f'../data/results/{benchmark_type}_{name}.pkl', 'wb') as f:
        pickle.dump((sols, routing_manager), f)
    del routing_manager, benchmark_type, name, sol, sols


In [4]:
from tqdm import tqdm
from loguru import logger as log
log.remove()
NUM_WORKERS = 4
MAX_SIZE = 10 # создаем столько менеджеров. потом в парралель NUM_WORKERS  решают

with Pool(NUM_WORKERS ) as p:
    for benchmark_type in ['pdp_100']:#os.listdir('../data/Li & Lim benchmark'):
        rms = []
        for name in tqdm(os.listdir(f'../data/Li & Lim benchmark/{benchmark_type}')):
            depo, cargos, tariff = parse_data(f'../data/Li & Lim benchmark/{benchmark_type}/{name}')
            rms.append((rm(depo, cargos, tariff, p), benchmark_type, name))
            if len(rms) == MAX_SIZE:
                rr = list(p.imap_unordered(rms_calc, rms))
                rms = []
        break
    if len(rms) > 0:
        rr = list(p.imap_unordered(rms_calc, rms))
        rms = []

 16%|█▌        | 9/56 [00:00<00:02, 19.62it/s]

12
114.7421582177598 75
1212

225.302753041037114.7421582177598  14375

12
212.008771254956912 143

12
12
12
12
12
12
12
228.0087712549569 190
12
12
12
12
12
435.82542442102664 352
12
435.82542442102664 352
12
225.83543093852057 79
12
225.83543093852057 79


 34%|███▍      | 19/56 [00:25<00:38,  1.05s/it]

12
218.80273550544842 41
12
218.80273550544842 41
12
12
12
12
12
12
12
12
12
12
184.3238075793812 126
12
12
12
12


 34%|███▍      | 19/56 [00:39<00:38,  1.05s/it]

...

 88%|████████▊ | 49/56 [01:50<00:08,  1.18s/it]

12
12
12
12
2363.04834939252 1910
12
12
281.56604182158395 265
12
12


100%|██████████| 56/56 [02:10<00:00,  2.33s/it]

1212
12
663.0
 347
1212



1212
567.0
12 
347
12
12
12
588.3851648071345 280
12
588.3851648071345 280


In [11]:
from vrp_study.pdptw_model.solution_checker import check_solution

for file_name in os.listdir('../data/graphs'):
    solution_name = file_name.replace('_cg','')
    with open(f'../data/results/{solution_name}','rb') as f:
        res = pickle.load(f)
    sols, rm = res
    rm: RoutingManager = rm
    assert check_solution(rm,sols)

In [12]:
import networkx as nx
for file_name in os.listdir('../data/graphs'):
    solution_name = file_name.replace('_cg','')
    print(file_name, solution_name)
    with open(f'../data/graphs/{file_name}','rb') as f:
        cg = pickle.load(f)
    with open(f'../data/results/{solution_name}','rb') as f:
        res = pickle.load(f)
    sols, rm = res
    rm: RoutingManager = rm
    cg: nx.DiGraph = cg
    # print(cg.edges())
    for sol in sols:
        cargo_path_set = set()
        cargo_path = []
        for i in range(len(sol)):
            pdp = rm.nodes()[sol[i]].pdp_id
            if pdp not in cargo_path_set:
                cargo_path_set.add(pdp)
                cargo_path.append(pdp)
        print(sol)
        print(cargo_path)
        for i in range(len(cargo_path)-1):
            p1 = cargo_path[i]
            p2 = cargo_path[i+1]
            n1 = rm.get_pick_up_and_delivery_nodes()[p1][0]
            n2 = rm.get_pick_up_and_delivery_nodes()[p2][0]
            e = cg.edges()[n1,n2]
            print(e)

pdp_100_lrc204.txt_cg.pkl pdp_100_lrc204.txt.pkl
[73, 95, 33, 51, 31, 85, 35, 32, 74, 29, 37, 34, 30, 27, 55, 96, 86, 36, 45, 53, 52, 38, 47, 43, 46, 49, 48, 56, 28, 54, 50, 44]
[36, 47, 16, 25, 15, 42, 17, 14, 18, 13, 27, 22, 26, 23, 21, 24]
{'length': np.float64(0.8228306552571969), 'l_ab': 35.35533905932738, 'l_cd': 31.953090617340916, 'l0': 67.3184296766683, 'l1': 54.19100840863323}
{'length': np.float64(0.8118694494267554), 'l_ab': 31.953090617340916, 'l_cd': 40.162171256046406, 'l0': 72.12526187338733, 'l1': 57.093222890176165}
{'length': np.float64(1.3162448212632774), 'l_ab': 40.162171256046406, 'l_cd': 31.78049716414141, 'l0': 71.95266842018782, 'l1': 91.72402771346543}
{'length': np.float64(1.5097151580698795), 'l_ab': 31.78049716414141, 'l_cd': 21.540659228538015, 'l0': 53.331156392679425, 'l1': 75.29937945140045}
{'length': np.float64(1.0464562958550805), 'l_ab': 21.540659228538015, 'l_cd': 22.360679774997898, 'l0': 43.91133900353591, 'l1': 45.90533094452347}
{'length': np.